<a href="https://colab.research.google.com/github/lauxpress/NLP_UBA_Scheihing23/blob/main/NLP_Desaf%C3%ADo_2_Scheihing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Bot "simple"

In [ ]:
import json
import string
import random
import re # Regular Expressions (regex)
import urllib.request

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Como soy altamente catolico, vamos a crear un bot que pueda respondernos preguntas sobre Dios.

In [ ]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Lionel_Messi')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [ ]:
article_text

'\n\nargentine professional footballer\neponyms and public art\nmedia\nfamily\nlionel andrés messi[note 1] (spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 june 1987), also known as leo messi, is an argentine professional footballer who plays as a forward for and captains both major league soccer club inter miami and the argentina national team. widely regarded as one of the greatest players of all time, messi has won a record eight ballon d\'or awards[note 2] and a record six european golden shoes, and in 2020 he was named to the ballon d\'or dream team. until leaving the club in 2021, he had spent his entire professional career with barcelona, where he won a club-record 34 trophies, including ten la liga titles, seven copa del rey titles and the uefa champions league four times.[note 3] with his country, he won the 2021 copa américa and the 2022 fifa world cup. a prolific goalscorer and creative playmaker, messi holds the records for most goals in la liga (474), most hat-tr

Realizamos el preprocesamiento, removemos caracteres especiales y quitamos los espacios y saltos.

In [ ]:
# substituir con regex con espacio vacío:
text = re.sub(r'\[[0-9]*\]', ' ', article_text) # substituir los números entre corchetes
# Substituir las anotaciones "[note X]"
text = re.sub(r'\[note\s[0-9]+\]', ' ', text)
# Substituir el símbolo ⓘ
text = re.sub(r'ⓘ', ' ', text)
# (notar que los corchetes son interpretados literalmente por los backlsash)
text = re.sub(r'\s+', ' ', text) # substituir más de un caracter de espacio, salto de línea o tabulación

In [ ]:
text

' argentine professional footballer eponyms and public art media family lionel andrés messi (spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 june 1987), also known as leo messi, is an argentine professional footballer who plays as a forward for and captains both major league soccer club inter miami and the argentina national team. widely regarded as one of the greatest players of all time, messi has won a record eight ballon d\'or awards and a record six european golden shoes, and in 2020 he was named to the ballon d\'or dream team. until leaving the club in 2021, he had spent his entire professional career with barcelona, where he won a club-record 34 trophies, including ten la liga titles, seven copa del rey titles and the uefa champions league four times. with his country, he won the 2021 copa américa and the 2022 fifa world cup. a prolific goalscorer and creative playmaker, messi holds the records for most goals in la liga (474), most hat-tricks in la liga (36) and the uef

Ahora vamos a dividir el texto en palabras y oraciones.

In [ ]:
corpus = nltk.sent_tokenize(text) # divide en oraciones
words = nltk.word_tokenize(text) # divide en términos

In [ ]:
print("Términos:", len(words))

Términos: 25626


Creamos las funciones para lematizar los tokens y borrar simbolos de puntuacion

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# ord() nos da el código Unicode para un caracter dado
punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula (string.lower())
    # 2 - quitar los simbolos de puntuacion (string.translate())
    # 3 - realiza la tokenización (nltk.word_tokenize)
    # 4 - realiza la lematización (nuestra función perform_lemmatization)
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

Utilizamos vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar

    corpus.remove(user_input)
    return response

In [ ]:
def bot_response(human_text):
    print("Q:", human_text)
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

A continuacion podemos ver varios ejemplos de las respuestas qeu ofrece este bot.

In [ ]:
bot_response('lionel andrés messi')

Q: lionel andrés messi


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


A: "congratulations on your historic record, lionel.


'"congratulations on your historic record, lionel.'

In [ ]:
bot_response(' Ballon dOr awards')

Q:  Ballon dOr awards
A: as the odds-on favourite, messi again won the fifa ballon d'or, becoming the only player in history to win the ballon d'or four times.


"as the odds-on favourite, messi again won the fifa ballon d'or, becoming the only player in history to win the ballon d'or four times."

In [ ]:
bot_response('what is leo messi')

Q: what is leo messi
A: i admire you very much, leo messi."


'i admire you very much, leo messi."'

In [ ]:
bot_response('UEFA Champions League')

Q: UEFA Champions League
A: on 18 october, in his 122nd european club appearance, messi scored his 97th uefa champions league goal, and his 100th in all uefa club competitions, in a 3–1 home victory over olympiacos.


'on 18 october, in his 122nd european club appearance, messi scored his 97th uefa champions league goal, and his 100th in all uefa club competitions, in a 3–1 home victory over olympiacos.'

In [ ]:
bot_response('argentina national team')

Q: argentina national team
A: you go around the world and people say, 'he's the top scorer for the argentina national team.'


"you go around the world and people say, 'he's the top scorer for the argentina national team.'"

In [ ]:
bot_response('perception of messi among argentines')

Q: perception of messi among argentines
A: vickery states the perception of messi among argentines changed in 2019, with messi making a conscious effort to become "more one of the group, more argentine", with vickery adding that following the world cup victory in 2022 messi would now be held in the same esteem by his compatriots as maradona.


'vickery states the perception of messi among argentines changed in 2019, with messi making a conscious effort to become "more one of the group, more argentine", with vickery adding that following the world cup victory in 2022 messi would now be held in the same esteem by his compatriots as maradona.'

Claramente esta tecnica no nos sirvio mucho, el hecho de que encuentre similitudes mayores en frases que nada tienen que ver con lo que estemos preguntando y no haya forma de contextualizar en profundidad al bot hace que no obtengamos nunca la informacion que solicitamos. Tambien podemos notar que el articulo parece tener comentarios o frases de otras personas citadas y estos producen las similitudes mas grandes con las requests. No se a que se deba esto de los comentarios pero claramente hacen que el bot no seleccione similitudes que quiza pudieran darnos mejores resultados.

Quiza cortando el corpus a la mitad no tendria lo que parece ser la seccion de comentarios o citaciones y asi bypassear esas frases que me dan respuestas incorrectas.

In [ ]:
# Encontrar la mitad del texto
mitad = len(article_text) // 2

# Reducir el texto a la mitad
texto_reducido = article_text[:mitad]

In [ ]:
# substituir con regex con espacio vacío:
text = re.sub(r'\[[0-9]*\]', ' ', texto_reducido) # substituir los números entre corchetes
# Substituir las anotaciones "[note X]"
text = re.sub(r'\[note\s[0-9]+\]', ' ', text)
# Substituir el símbolo ⓘ
text = re.sub(r'ⓘ', ' ', text)
# (notar que los corchetes son interpretados literalmente por los backlsash)
text = re.sub(r'\s+', ' ', text) # substituir más de un caracter de espacio, salto de línea o tabulación

In [ ]:
text

' argentine professional footballer eponyms and public art media family lionel andrés messi (spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 june 1987), also known as leo messi, is an argentine professional footballer who plays as a forward for and captains both major league soccer club inter miami and the argentina national team. widely regarded as one of the greatest players of all time, messi has won a record eight ballon d\'or awards and a record six european golden shoes, and in 2020 he was named to the ballon d\'or dream team. until leaving the club in 2021, he had spent his entire professional career with barcelona, where he won a club-record 34 trophies, including ten la liga titles, seven copa del rey titles and the uefa champions league four times. with his country, he won the 2021 copa américa and the 2022 fifa world cup. a prolific goalscorer and creative playmaker, messi holds the records for most goals in la liga (474), most hat-tricks in la liga (36) and the uef

In [ ]:
corpus = nltk.sent_tokenize(text) # divide en oraciones
words = nltk.word_tokenize(text) # divide en términos

In [ ]:
print("Términos:", len(words))

Términos: 13057


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar

    corpus.remove(user_input)
    return response

In [ ]:
bot_response('lionel andrés messi')

Q: lionel andrés messi
A: "congratulations on your historic record, lionel.


'"congratulations on your historic record, lionel.'

In [ ]:
bot_response('argentina national team')

Q: argentina national team
A: ahead of the new season, a major concern remained his frequent muscular injuries, which had left him side-lined for a total of eight months between 2006 and 2008. to combat the problem, the club implemented new training, nutrition, and lifestyle regimens, and assigned him a personal physiotherapist, who would travel with him during call-ups for the argentina national team.


'ahead of the new season, a major concern remained his frequent muscular injuries, which had left him side-lined for a total of eight months between 2006 and 2008. to combat the problem, the club implemented new training, nutrition, and lifestyle regimens, and assigned him a personal physiotherapist, who would travel with him during call-ups for the argentina national team.'

Bueno, como se esperaba, reducir el texto a la mitad cambio el resultado de la respuesta del bot, pero sigue sin ser una respuesta correcta.
Se concluye que este metodo no es factible para este tipo de implementaciones con una pagina entera de wikipedia.